# LangChain 实战：CargoSmart 聊天客服

## 使用 GPT-4 构造聊天话术数据

使用 ChatGPT 构造聊天数据的 Prompt：

```
你是 CargoSmart 公司的客服专员，现在培训职场新人，请给出 100 条实用的客户回复话术。

每条回复话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
以下是针对不同客户问题的20条实用客户回复话术，这些话术有助于提升客户服务体验并有效解决问题：

[客户问题] 我的货物迟到了，这会严重影响我的业务，你们能做些什么？
[销售回答] 我非常理解您的紧急情况，对于造成的不便我深表歉意。我将立即查看具体情况，并尽快向您反馈可能的最快解决方案。

[客户问题] 我可以在哪里查看我的货物状态？
[销售回答] 您可以通过我们的在线追踪系统查看您货物的最新状态，我也可以帮您查询并提供详细信息。

[客户问题] 你们的价格比其他公司高，为什么我还应该选择你们？
[销售回答] 我们的服务确实在市场上属于优质范畴，我们提供的不仅是运输，还包括全方位的客户支持和高效的问题解决方案，这在长远来看将为您节省更多成本和时间。

[客户问题] 如果我想更改订单怎么办？
[销售回答] 更改订单非常简单。您只需要告诉我具体的更改需求，我会立即为您处理。

[客户问题] 你们提供哪些类型的运输服务？
[销售回答] 我们提供多种运输服务，包括海运、空运和陆运等，根据您的具体需求，我们可以提供最合适的运输解决方案。

[客户问题] 我可以取消订单吗？
[销售回答] 是的，您可以取消订单。请告诉我您的订单号，我将帮助您处理取消流程。

[客户问题] 我的货物受损了，我该怎么办？
[销售回答] 对于您的货物受损，我们感到非常抱歉。请您提供损坏的详细信息和照片，我们将启动理赔流程，尽快为您解决问题。

[客户问题] 你们的服务有哪些保障措施？
[销售回答] 我们为所有服务提供保障措施，包括货物保险、实时追踪以及24小时客户服务支持，确保您的货物安全及时到达。

[客户问题] 我怎样才能得到更好的运费报价？
[销售回答] 我们可以根据您的货物量和运输需求提供个性化报价。请告诉我更多关于您的需求，我将为您计算最优惠的价格。

[客户问题] 你们的运输时间是多久？
[销售回答] 运输时间根据不同的目的地和运输方式会有所不同。请告诉我具体的目的地，我将为您提供一个预估的时间表。
```

## 使用 Document Transformers 模块来处理原始数据

将 ChatGPT 生成的结果保存到 [customer_service_specialist_data](customer_service_specialist_data) 文件中

In [1]:
with open("customer_service_specialist_data.txt", "r", encoding="utf-8") as f:
    customer_service_specialist_data = f.read()

In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 120,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([customer_service_specialist_data])

In [5]:
docs[0]

Document(page_content='[客户问题] 我的货物迟到了，这会严重影响我的业务，你们能做些什么？\n[销售回答] 我非常理解您的紧急情况，对于造成的不便我深表歉意。我将立即查看具体情况，并尽快向您反馈可能的最快解决方案。')

In [6]:
len(docs)

80

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

E:\code\edward\openai-quickstart\.venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
query = "如何确保我的隐私和交易安全？"

In [9]:
answer_list = db.similarity_search(query)

In [10]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 如何确保我的隐私和交易安全？
[销售回答] 保护客户的隐私和交易安全是我们的首要任务。我们使用加密技术来保护所有的交易信息，并严格遵守数据保护法规，确保您的信息安全。

[客户问题] 如何确保敏感货物的隐私和安全？
[销售回答] 对于敏感货物，我们采取额外的保密和安全措施，包括但不限于专用运输和加密的数据处理。

[客户问题] 如何能保证我的货物在运输中不会被偷或丢失？
[销售回答] 我们采用多重安全措施保护货物，包括密封性良好的包装、安全的运输方式和全程监控系统。

[客户问题] 你们的服务有哪些保障措施？
[销售回答] 我们为所有服务提供保障措施，包括货物保险、实时追踪以及24小时客户服务支持，确保您的货物安全及时到达。



In [11]:
db.save_local("customer_service_specialist_data")